<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Table of Contents</a></span></li><li><span><a href="#Recurrent-Neural-Networks" data-toc-modified-id="Recurrent-Neural-Networks-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Recurrent Neural Networks</a></span><ul class="toc-item"><li><span><a href="#Long-Short-Term-Memory" data-toc-modified-id="Long-Short-Term-Memory-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Long Short Term Memory</a></span></li><li><span><a href="#LSTM-in-Keras" data-toc-modified-id="LSTM-in-Keras-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LSTM in Keras</a></span></li><li><span><a href="#Textaufbereitung-für-ein-RNN" data-toc-modified-id="Textaufbereitung-für-ein-RNN-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Textaufbereitung für ein RNN</a></span><ul class="toc-item"><li><span><a href="#Keras-embedding-layer:" data-toc-modified-id="Keras-embedding-layer:-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Keras embedding layer:</a></span></li></ul></li><li><span><a href="#Beispiel:-Newsgroups-from-scratch" data-toc-modified-id="Beispiel:-Newsgroups-from-scratch-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Beispiel: Newsgroups from scratch</a></span></li><li><span><a href="#Hausaufgaben" data-toc-modified-id="Hausaufgaben-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Hausaufgaben</a></span></li></ul></li></ul></div>

# Recurrent Neural Networks

Die Convolutional Neural Networks (CNN) sind sehr gut darin, statische Muster in Daten zu erkennen. Sie berücksichtigen allerdings keine Strukturen in der Zeit, Strukturen, wie sie typisch sind für die natürliche Sprache oder Videos. 

Grundidee der Recurrent NN: Es gibt eine Schleife über die Inputdaten, die jeweils einen Zeitschritt repräsentieren, z.B. jeweils ein Wort in einem Satz oder ein Bild in einem Video. Zugleich gibt es eine Zustand (state) der erinnert und durch den Input aktualisiert und modifiziert wird. 
<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-rolled.png" height=120 width=120 />

Das obenstehende Bild ist eine Zusammenfassung der Schleife, die ein RNN ausmacht. Wir können uns das auch etwas genauer anschauen, nämlich ausgerollt:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" width=440 />

Die Eingabe x<sub>t</sub> kann z.B. eine Folge von Worten sein. Dann ist x<sub>0</sub> das erste Wort, x<sub>1</sub> das zweite Wort bis hin zum letzten Wort x<sub>t</sub>. Die waagrechten Pfeile repräsentieren den Informationsfluss der über den Zustand geschieht.

Chollet beschreibt dies ganz gelungen mit diesem Pseudocode (Chollet 2017: 196):

    state_t = 0                          #the state at t
    for input_t in input_sequence:       #Iterates over sequence elements   
        output_t = f(input_t, state_t)   #f is a function, input is input_t and state_t 
        state_t = output_t               #The previous output becomes the state for the next iteration. 

        



Die Funktion f ist die typische Matrixmultiplikation auf die eine Aktivierungsfunktion gesetzt wird. Man kann das also, frei nach Chollet (2017: 197) als Pseudocode so schreiben:


    def recurrent_layer(input_sequence, W, U, b)
        state_t = [0,...,0]        #len(state_t) == len(output_t)
        successive_outputs = []    #output states at timestep t=0 to t=t
        for input_t in input_sequence:
            output_t = activation(dot(W, input_t) + dot(U, state_t) + b )
            successive_outputs.append(output_t)   
            state_t = output_t
        return successive_outputs
    
activation(): Aktivierungsfunktion, z.B. sigmoid, tanh usw.<br/>
dot(): Matrixmultiplikation (wie beim Dense-Layer)<br/>
W: Matrix mit Gewichten für den Input<br/>
U: Matrix mit Gewichten für den Zustand<br/>
b: bias 
successive_outputs = Liste, die am Ende alle Ausgaben für alle Eingaben zu allen Zeitschritten enthält

Wir können nun also noch einmal einen Blick auf die obenstehende Grafik werfen und zoomen diesmal sozusagen in den Kasten A und die waagrechten Pfeile hinein: 
<img src="rnn_1.png"/>

Ein grundlegendes Problem eines solche einfachen RNNs besteht darin, dass es zwar Information über die Zeit erhalten kann, aber leider nicht sehr lange. Offensichtlich wird bei jedem Zeitschritt eine Matrixmultiplikation vorgenommen. Das kann zwei Effekte haben. Erstens gehen kleine Werte, die häufig mit kleinen Werten multipliziert werden, sehr schnell gegen 0. Wenn wir etwa Sätze mit 100 Worten untersuchen wollen, dann werden wir, selbst wenn wir immer nur 0.9 * 0.9 multiplizieren, am Ende einen sehr kleinen Wert haben:

In [8]:
0.9**100

2.6561398887587544e-05

Diesen Effekt nennt man den *vanishing gradient*. Die Kehrseite dieses Effekts ist der *exploding gradient*. Selbst wenn man relativ kleine Zahlen sehr häufig miteinander multipliziert, dann werden sie schnell sehr groß:

In [2]:
1.7**100

1.1088993727807807e+23

Beides führt dazu, dass einfache RNNs für die Praxis nicht geeignet sind. Vielmehr verwendet man zumeist eine Variante, die ausdrücklich dafür geschaffen wurde, das vanishing gradient-Problem zu lösen: die *Long Short Term Memory*. 

## Long Short Term Memory
LSTMs haben sich in der Praxis etabliert, weil sie das Problem der *vanishing* bzw. *exploding gradients* im Griff haben. Ausgerollt kann man sie sich so vorstellen ![Chollet Bild 6.15](chollet_6-15.png)

Die Berechnung des neuen *carry*-Werts geschieht über drei weitere Matrixmultiplikationen. Wiederum in Chollets Pseudocode:

    output_t = activation(dot(state_t, Uo) + dot(input_t, Wo) + dot(C_t, Vo) + bo)
    i_t = activation(dot(state_t, Ui) + dot(input_t, Wi) + bi) 
    f_t = activation(dot(state_t, Uf) + dot(input_t, Wf) + bf) 
    k_t = activation(dot(state_t, Uk) + dot(input_t, Wk) + bk)
    c_t+1 = i_t * k_t + c_t * f_t

In jedem Zeitschritt bzw. bei jedem neuen Wort eines Satzes werden also vier Matrixmultiplikationen vorgenommen! In der Literatur werden diese zusätzlichen Operationen zumeist als *Gate* bezeichnet. Man spricht dann von *input gate*, *forget gate*, und *output gate*. 

![Darstellung LSTM](lstm1.png)
![Legende](lstm_legende.png)
(Bildquelle: http://colah.github.io/posts/2015-08-Understanding-LSTMs/)



## LSTM in Keras 

Wie üblich ist die tatsächliche Verwendung von LSTMs in Keras sehr einfach

In [1]:
from keras.layers import LSTM
from keras import models

model = models.Sequential()
model.add(LSTM(64))   #64 = Anzahl der units



Using TensorFlow backend.


## Textaufbereitung für ein RNN

Der erste Schritt für die Arbeit mit RNNs ist die Aufbereitung der Textdaten. Dieser Schritt besteht wiederum aus zwei Teilschritten. Im ersten segmentieren wir den Text in gleich große Texteinheiten, dann ersetzen wir, wie üblich, die symbolische Repräsentation, also die Worte, durch Zahlen. 

1) Textsegmentierung
Wie wir bei diesem Schritt vorgehen, hängt sehr von unserer Fragestellung und Korpus ab. Wenn wir etwa Tweets klassifizieren wollen, dann können wir einfach 140 bzw. 280 Zeichen lange Einheiten definieren. Was der aktuelle Text davon nicht verwendet, füllen wir dann mit einem padding-Zeichen auf. Wenn wir längere Texte haben, können wir diese entweder in kleinere Blöcke zerlegen und dann die Klassifikation für jeden Block durchführen (und am Ende ein Voting über alle Blöcke eines Textes machen) oder wir samplen einen Textabschnitt aus dem Text bzw. nehmen einfach immer einen gleich langen Abschnitt am Anfang. Wie immer wir vorgehen, am Ende dieses Schritts haben wir einen lauter gleich lange Textsegmente, so dass unsere Trainingsdaten dann aus einer Liste dieser Segmente bestehen.

2) Wort->Zahl

Bei diesem Schritt ersetzen wir die Worte des Textes durch einen Vektor. Es gibt im wesentlichen drei Ansätze. Wir werden dabei übrigens immer von Tokens sprechen. Tokens sind beliebige gleichförmige sprachliche Einheiten, also z.B. Buchstaben, Worte, Buchstaben-Ngramme oder Wort-Ngramme. Wir denken aber zuerst einmal nur an Worte. 
* One-Hot-Encoding der Tokens. Nachteil: Vektor für jedes Token ist so groß wie der Wortschatz und die Encodings haben keine brauchbare Bedeutung, so dass sinnverwandte Worte genausoweit von einander entfernt sind wie sinnferne.
* Word Embeddings, die direkt auf den Trainingsdaten berechnet werden. Vorteil: Sehr einfach, Nachteil: das Trainingskorpus muss relativ groß sein, um gute Ergebnisse zu erzielen. Außerdem sind moderne Embeddings deutlich raffinierter als das Keras-Embedding. Vortrainierte Embeddings können zudem umfassend Sprachinformationen einbringen, die die Verarbeitungsergebnisse sehr verbessern.
* Vortrainierte Embeddings, die auf sehr großen Trainingskorpora berechnet wurden. Hier gibt es verschiedene Techniken: 

|.|word2vec | glove |fasttext |elmo  | bert|
|---|---|---|---|---|---|
|Entstehungsjahr |2013  |2014|2016|2018|2018|
|deutsche Modelle |x  |x  |x  |  x|x|
|oov*|o  |o  |x  |x |x |
|kontextsensitiv|o  |o  |o  |x|x|

*oov: out of vocabulary

Für viele Zwecke sind die FastText-Modelle ein guter Ausgangspunkt, da sie für viele Sprachen vorliegen, gut mit oov-Wörtern umgehen und sich leicht weitertrainieren lassen.  


One-Hot-Encoding schauen wir uns nicht an, da unpraktisch. Beschreibung bei Chollet. 

### Keras embedding layer:

In [3]:
from keras.layers import Embedding 

vocab_size = 10000 #Wieviele Wörter (der häufigsten Wörter) werden überhaupt verwendet
embed_size = 96    #Wie groß ist der Vektor für jedes Wort, je nach Korpus 
                   #zwischen 64 und 1024
embedding_layer = Embedding(vocab_size, embed_size)

## Beispiel: Newsgroups from scratch
Quelle: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

Code: https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py

In [2]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Embedding, Flatten



In [4]:

TEXT_DATA_DIR = "20_newsgroup"

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 19997 texts.


In [5]:
MAX_NB_WORDS=20000
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
VALIDATION_SPLIT = 0.2


tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

e:\conda\envs\keras\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 174074 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [14]:
GLOVE_DIR = "glove_dir"

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [7]:
EMBEDDING_DIM = 100  # dimension der glove vektoren, die wir verwenden

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [8]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [17]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=20, batch_size=128)


Train on 15998 samples, validate on 3999 samples
Epoch 1/20
15998/15998 [==============================] - 4s 248us/step - loss: 2.3800 - acc: 0.2178 - val_loss: 1.9144 - val_acc: 0.3298
Epoch 2/20
15998/15998 [==============================] - 3s 194us/step - loss: 1.5427 - acc: 0.4624 - val_loss: 1.3459 - val_acc: 0.5069
Epoch 3/20
15998/15998 [==============================] - 3s 194us/step - loss: 1.1901 - acc: 0.5901 - val_loss: 1.1320 - val_acc: 0.6159
Epoch 4/20
15998/15998 [==============================] - 3s 195us/step - loss: 0.9653 - acc: 0.6709 - val_loss: 0.9897 - val_acc: 0.6744
Epoch 5/20
15998/15998 [==============================] - 3s 194us/step - loss: 0.8152 - acc: 0.7223 - val_loss: 0.9207 - val_acc: 0.6947
Epoch 6/20
15998/15998 [==============================] - 3s 194us/step - loss: 0.7098 - acc: 0.7619 - val_loss: 0.8645 - val_acc: 0.7137
Epoch 7/20
15998/15998 [==============================] - 3s 195us/step - loss: 0.5902 - acc: 0.8014 - val_loss: 1.0458 - v

In [16]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = LSTM(128)(embedded_sequences)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
history = model.fit(x_train, y_train, 
                    validation_data=(x_val, y_val),
                    epochs=6, 
                    batch_size=128)

Train on 15998 samples, validate on 3999 samples
Epoch 1/6
15998/15998 [==============================] - 219s 14ms/step - loss: 2.7339 - acc: 0.1411 - val_loss: 2.4380 - val_acc: 0.1940
Epoch 2/6
15998/15998 [==============================] - 219s 14ms/step - loss: 2.2833 - acc: 0.2415 - val_loss: 2.2338 - val_acc: 0.2591
Epoch 3/6
15998/15998 [==============================] - 217s 14ms/step - loss: 1.9498 - acc: 0.3416 - val_loss: 1.8453 - val_acc: 0.3626
Epoch 4/6
15998/15998 [==============================] - 218s 14ms/step - loss: 1.6960 - acc: 0.4221 - val_loss: 1.5178 - val_acc: 0.4749
Epoch 5/6
15998/15998 [==============================] - 210s 13ms/step - loss: 1.4494 - acc: 0.5006 - val_loss: 1.3540 - val_acc: 0.5466
Epoch 6/6
15998/15998 [==============================] - 210s 13ms/step - loss: 1.2729 - acc: 0.5652 - val_loss: 1.2291 - val_acc: 0.5916


## Hausaufgaben

1) Nachbauen
2) Parameter-Suche: 
a) Optimierung von CNN
b) Optimierung des LSTMs (2 Layer mit Dropout dazwischen, Bidirektional)